In [46]:
import ee
import geemap
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import ipynb
import time
from speckle_filter import leesigma  # for lee sigma speckle filter

%matplotlib inline

In [47]:
# Get authentication credentials to initialize ee
#ee.Authenticate()
ee.Initialize()

In [48]:
# Define a region of interest
roi = ee.Geometry.Rectangle([70.7980547394194417, 23.2880385161501735, 71.5060515087354958, 23.9024834537972986])  # Nanda Bet small region
roi_s1 = ee.Geometry.Rectangle([69.98568, 22.66953, 72.76651, 24.60271])  # Nanda Bet S1 extent (different from boundary)
roi_l8 = ee.Geometry.Point([71.98385483, 23.23565591]) # right -- this does not get all right side images
 

In [49]:
# Load boundary shapefiles
shp_file = 'data/s1_boundary.shp'
s1_boundary = geemap.shp_to_ee(shp_file)

shp_file = 'data/boundary_for_landsat.shp'
boundary_for_landsat = geemap.shp_to_ee(shp_file)

landsat_point = ee.Geometry.Point([72.03943080, 23.91425079]) # this will get landsat tracks on the right side of s1 extent

In [50]:
type(ee.Geometry.Polygon(s1_boundary))


ee.geometry.Geometry

In [51]:
# functions for db to linear and vice versa
def db2pow(db_val):
    return 10**(db_val/10)

def pow2db(pow_val):
    return 10*math.log(pow_val,10)

In [52]:
# Import S1 data
d1 = '2020-01-01'
d2 = '2021-01-01'
img_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filterMetadata('resolution_meters', 'equals', 10)
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    .filterBounds(roi)
    .filterDate(d1, d2))
    
n = img_collection.size().getInfo()    
print('Number of images in collection:', n)
print('Bands:', img_collection.first().bandNames().getInfo())

Number of images in collection: 31
Bands: ['VV', 'VH', 'angle']


In [53]:
# Get dates of all images in image collection
img_collection_list = img_collection.toList(n)
# date_list = []
# for i in range(0, n):
#     img = ee.Image(img_collection_list.get(i))
#     t_ms = img.get('system:time_start').getInfo()
#     dt = datetime.fromtimestamp(t_ms/1000.0)
#     d = dt.strftime("%m/%d/%Y")
#     date_list.append(d)

# print(date_list)

acq_times = img_collection.aggregate_array('system:time_start').getInfo()
date_list = [time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times]
print(date_list)

['01-03-2020', '01-15-2020', '01-27-2020', '02-08-2020', '02-20-2020', '03-03-2020', '03-15-2020', '03-27-2020', '04-08-2020', '04-20-2020', '05-02-2020', '05-14-2020', '05-26-2020', '06-07-2020', '06-19-2020', '07-01-2020', '07-13-2020', '07-25-2020', '08-06-2020', '08-18-2020', '08-30-2020', '09-11-2020', '09-23-2020', '10-05-2020', '10-17-2020', '10-29-2020', '11-10-2020', '11-22-2020', '12-04-2020', '12-16-2020', '12-28-2020']


In [54]:
print(img_collection.first().getInfo())

{'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [28784, 21894], 'crs': 'EPSG:32642', 'crs_transform': [10, 0, 599840.9106613587, 0, -10, 2726254.8468074747]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [28784, 21894], 'crs': 'EPSG:32642', 'crs_transform': [10, 0, 599840.9106613587, 0, -10, 2726254.8468074747]}, {'id': 'angle', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [21, 10], 'crs': 'EPSG:32642', 'crs_transform': [-12721.525722365594, -3302.3478144576075, 883305.2359083665, 2120.651907443069, -20203.65323428344, 2679278.5895491294]}], 'id': 'COPERNICUS/S1_GRD_FLOAT/S1A_IW_GRDH_1SDV_20200103T011003_20200103T011028_030629_038274_4C20', 'version': 1578144077383526, 'properties': {'SNAP_Graph_Processing_Framework_GPF_vers': '6.0.4', 'SLC_Processing_facility_org': 'ESA', 'SLC_Processing_facility_country': 'Germany', 'GRD_Post_Processing_facility_org': 'E

In [55]:
# Define visualization parameters
vv_vis = {
    'min': 0,
    'max': 0.3,
    'palette': ['#000000','#ffffff']
}

vv_sqrt_vis = {
    'min': np.sqrt(vv_vis.get('min')),
    'max': np.sqrt(vv_vis.get('max')),
    'palette': ['#000000','#ffffff']
}

vh_vis = {
    'min': 0.001,
    'max': 0.03,
    'palette': ['#000000','#ffffff']
}

vh_sqrt_vis = {
    'min': np.sqrt(vh_vis.get('min')),
    'max': np.sqrt(vh_vis.get('max')),
    'palette': ['#000000','#ffffff']
}

vv_vh_vis = {
    'min': 3,
    'max': 30,
    'palette': ['#000000','#ffffff']
}

# Select individual images to display on map
idx = 1
vv = ee.Image(img_collection_list.get(idx)).select('VV')
vh = ee.Image(img_collection_list.get(idx)).select('VH')
vv_vh = vv.divide(vh)

# Implement speckle filter on images
kernel = 5  # kernel size
vv_filtered = leesigma(vv,kernel)
vh_filtered = leesigma(vh,kernel)
vv_vh_filtered = vv_filtered.divide(vh_filtered)

# Get sqrt of backscatter
vv_sqrt = vv_filtered.sqrt()
vh_sqrt = vh_filtered.sqrt()

print(date_list[idx])

01-15-2020


In [56]:
# Create map and add layers

M = geemap.Map()
M.centerObject(roi, 8)
#M.addLayer(vv,vv_vis,'VV')
M.addLayer(vv_filtered,vv_vis,'VV with speckle filter')
#M.addLayer(vv_sqrt,vv_sqrt_vis,'VV sqrt')
#M.addLayer(vh,vh_vis,'VH')
M.addLayer(vh_filtered,vh_vis,'VH with speckle filter')
#M.addLayer(vh_sqrt,vh_sqrt_vis,'VH sqrt')
#M.addLayer(vv_vh,vv_vh_vis,'VV/VH')
M.addLayer(vv_vh_filtered,vv_vh_vis,'VV/VH with speckle filter')

# left_layer = geemap.ee_tile_layer(vh, vh_vis, 'VV')
# right_layer = geemap.ee_tile_layer(vh_filtered, vh_vis, 'VV Filtered')

# M.split_map(left_layer, right_layer)
M

Map(center=[23.595422252032954, 71.15205312407748], controls=(WidgetControl(options=['position', 'transparent_…

In [57]:
# Import Landsat8
d1 = '2020-01-01'
d2 = '2021-01-01'
l8_collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(boundary_for_landsat)
    .filterDate(d1, d2)
    .filter(ee.Filter.lte('CLOUD_COVER',10)))

n2 = l8_collection.size().getInfo()    
print('Number of images in Landsat8 collection:', n2)
print('Bands:', l8_collection.first().bandNames().getInfo())

acq_times = l8_collection.aggregate_array('system:time_start').getInfo()
date_list_l8 = [time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times]
print('Dates:', date_list_l8)

Number of images in Landsat8 collection: 51
Bands: ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']
Dates: ['01-14-2020', '01-30-2020', '03-02-2020', '03-18-2020', '04-03-2020', '04-19-2020', '05-05-2020', '05-21-2020', '10-12-2020', '10-28-2020', '11-29-2020', '12-15-2020', '12-31-2020', '01-14-2020', '01-30-2020', '03-02-2020', '03-18-2020', '04-03-2020', '04-19-2020', '05-05-2020', '05-21-2020', '06-06-2020', '10-12-2020', '10-28-2020', '11-29-2020', '12-15-2020', '12-31-2020', '01-05-2020', '01-21-2020', '02-06-2020', '02-22-2020', '04-10-2020', '04-26-2020', '05-12-2020', '05-28-2020', '07-15-2020', '10-03-2020', '11-20-2020', '12-06-2020', '01-05-2020', '01-21-2020', '02-06-2020', '02-22-2020', '03-09-2020', '04-10-2020', '04-26-2020', '05-12-2020', '05-28-2020', '10-03-2020', '11-20-2020', '12-06-2020']


In [32]:
# Clip Landsat data to S1 boundary, which is a feature collection
def clip_to_fc(img): 
    img = img.clipToCollection(s1_boundary)
    return img

l8_collection = l8_collection.map(clip_to_fc)

In [33]:
# Get NDWI from Landsat8

def get_ndwi_l8(img):
    ndwi = img.normalizedDifference(['SR_B5','SR_B6']).rename('NDWI')
    return ndwi

def get_mndwi_l8(img):
    mndwi = img.normalizedDifference(['SR_B3','SR_B6']).rename('MNDWI')
    return mndwi

def get_ndvi_l8(img):
    ndvi = img.normalizedDifference(['SR_B5','SR_B4']).rename('NDVI')
    return ndvi

ndwi_l8 = l8_collection.map(get_ndwi_l8)
mndwi_l8 = l8_collection.map(get_mndwi_l8)
ndvi_l8 = l8_collection.map(get_ndvi_l8)

ndwi_l8_list = ndwi_l8.toList(n2)
mndwi_l8_list = mndwi_l8.toList(n2)
ndvi_l8_list = ndvi_l8.toList(n2)

In [37]:
# Display on map

ndwi_vis = {
  'min': -0.25,
  'max': 0.25,
  #'palette': ['#0000ff', '#00ffff', '#ffff00', '#ff0000', '#ffffff'],
  'palette': ['#ffffff', '#ff0000', '#ffff00', '#00ffff', '#0000ff'],
}

ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['FF0000','FFFFFF','00FF00']
}

idx = 0
k1 = 13
k2 = 27
k3 = 39
print(date_list_l8[idx])
print(date_list_l8[idx+k1])
print(date_list_l8[idx+k2])
print(date_list_l8[idx+k3])

M = geemap.Map()
M.centerObject(roi, 8)
M.add_basemap('HYBRID')
#M.addLayer(vv_filtered,vv_vis,'VV with speckle filter')
#M.addLayer(vv_sqrt,vv_sqrt_vis,'VV sqrt')
#M.addLayer(vh,vh_vis,'VH')
#M.addLayer(vh_filtered,vh_vis,'VH with speckle filter')
#M.addLayer(vh_sqrt,vh_sqrt_vis,'VH sqrt')
#M.addLayer(vv_vh,vv_vh_vis,'VV/VH')
#M.addLayer(vv_vh_filtered,vv_vh_vis,'VV/VH with speckle filter')
#M.addLayer(vv_filtered,vv_vis,'VV with speckle filter')
#M.addLayer(ee.Image(ndwi_l8_list.get(idx)),ndwi_vis,'NDWI')
#M.addLayer(ee.Image(ndwi_l8_list.get(idx+k)),ndwi_vis,'NDWI 2')
M.addLayer(ee.Image(mndwi_l8_list.get(idx)),ndwi_vis,'MNDWI')
M.addLayer(ee.Image(mndwi_l8_list.get(idx+k1)),ndwi_vis,'MNDWI 2')
M.addLayer(ee.Image(mndwi_l8_list.get(idx+k2)),ndwi_vis,'MNDWI 3')
M.addLayer(ee.Image(mndwi_l8_list.get(idx+k3)),ndwi_vis,'MNDWI 4')
#M.addLayer(ee.Image(ndvi_l8_list.get(idx)),ndvi_vis,'NDVI')
M.addLayer(s1_boundary,{},'boundary')
M  # show map


01-14-2020
01-14-2020
01-05-2020
01-05-2020


Map(center=[23.595422252032954, 71.15205312407748], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# plot ndwi histogram

hist_ndwi = ee.Image(ndwi_l8_list.get(idx)).reduceRegion(
        ee.Reducer.fixedHistogram(-0.5, 0.5, 5000),roi,maxPixels=10000000).get('NDWI').getInfo()
a = np.array(hist_ndwi)
x = a[:, 0]                 # array of bucket edge positions
y = a[:, 1]/np.sum(a[:, 1]) # normalized array of bucket contents

# Create 3 subplots for histograms
f, ax1 = plt.subplots(1, 1, figsize=[15,6])
ax1.grid()
ax1.plot(x, y, '.')
ax1.set_ylabel('Normalized Count')
ax1.set_title(date_list_l8[idx] + ' - NDWI')

plt.show()

In [103]:
# Print image metadata

import pprint
i = 1
print(date_list_l8[i])
img = ee.Image(mndwi_l8_list.get(i)).select(['MNDWI']).clipToCollection(s1_boundary)
pprint.pprint(img.getInfo())

03-02-2020
{'bands': [{'crs': 'EPSG:32642',
            'crs_transform': [30, 0, 680085, 0, -30, 2673615],
            'data_type': {'max': 1,
                          'min': -1,
                          'precision': 'float',
                          'type': 'PixelType'},
            'dimensions': [7521, 7681],
            'id': 'MNDWI'}],
 'properties': {'system:index': 'LC08_149044_20200302'},
 'type': 'Image'}


In [38]:
print(n2)

51


In [39]:
# export landsat mndwi

# try this to remove nans before exporting:  ee.Filter.notNull

#print(ee.Image(mndwi_l8_list.get(0)).select('MNDWI').projection().crs().getInfo())
ns = ee.Image(mndwi_l8_list.get(0)).select('MNDWI').projection().nominalScale().getInfo()
print(ns)
task_id = []

for i in range(0,n2):
    img = ee.Image(mndwi_l8_list.get(i)).select(['MNDWI']).clipToCollection(s1_boundary)
    txt = 'l8_mndwi_nanda_bet_' + date_list_l8[i]
    task = ee.batch.Export.image.toDrive(image=img,
                                        description=txt,
                                        scale=ns,
                                        fileNamePrefix=txt,
                                        crs='EPSG:4326',
                                        fileFormat='GeoTIFF',
                                        maxPixels=1e9,
                                        region=s1_boundary.geometry())
    task.start()
    task_id.append(task.id)
    print(date_list_l8[i])

30
01-14-2020
01-30-2020
03-02-2020
03-18-2020
04-03-2020
04-19-2020
05-05-2020
05-21-2020
10-12-2020
10-28-2020
11-29-2020
12-15-2020
12-31-2020
01-14-2020
01-30-2020
03-02-2020
03-18-2020
04-03-2020
04-19-2020
05-05-2020
05-21-2020
06-06-2020
10-12-2020
10-28-2020
11-29-2020
12-15-2020
12-31-2020
01-05-2020
01-21-2020
02-06-2020
02-22-2020
04-10-2020
04-26-2020
05-12-2020
05-28-2020
07-15-2020
10-03-2020
11-20-2020
12-06-2020
01-05-2020
01-21-2020
02-06-2020
02-22-2020
03-09-2020
04-10-2020
04-26-2020
05-12-2020
05-28-2020
10-03-2020
11-20-2020
12-06-2020


In [45]:
task.list()

ProtocolError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [74]:
# Cancel tasks

# cancel last task
# task.cancel()

# cancel all tasks
for t in task_id:
    ee.data.cancelTask(t)

In [ ]:
# Print task id's
task_id

In [41]:
# Get task status
t_id = "4DQBDL6Z36AWI3TTM5OUGHUO"
print(ee.data.getTaskStatus(t_id))

t = ee.data.getTaskStatus(t_id)[0].get('creation_timestamp_ms')
print('\nTask started on:', time.strftime('%m-%d-%Y', time.gmtime(t/1000)))

[{'state': 'COMPLETED', 'description': 'l8_mndwi_nanda_bet_01-14-2020', 'creation_timestamp_ms': 1664846382162, 'update_timestamp_ms': 1664846680245, 'start_timestamp_ms': 1664846395790, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/'], 'attempt': 1, 'batch_eecu_usage_seconds': 218.1868133544922, 'id': '4DQBDL6Z36AWI3TTM5OUGHUO', 'name': 'projects/earthengine-legacy/operations/4DQBDL6Z36AWI3TTM5OUGHUO'}]

Task started on: 10-04-2022


In [ ]:
i = 0
print(ee.Image(mndwi_l8_list.get(i)).getInfo())

In [ ]:
print(7732*7881)

In [ ]:
for i in range(0,n2):
        print(date_list_l8[i])

In [ ]:
# Get Sentinel-2 NDWI and NDVI
s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate(d1, d2)
    .filterBounds(roi)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)))

n3 = s2_collection.size().getInfo()
s2_collection_list = s2_collection.toList(n3);  
print('Number of images in Sentinel-2 collection:', n3)

In [ ]:
# Get dates of each image
date_list_s2 = []
for i in range(0, n3):
    img = ee.Image(s2_collection_list.get(i))
    t_ms = img.get('system:time_start').getInfo()
    dt = datetime.fromtimestamp(t_ms/1000.0)
    d = dt.strftime("%m/%d/%Y")
    date_list_s2.append(d)

print(date_list_s2)

In [ ]:
false_color_vis = {
  'min': 0.0,
  'max': 3000,
  # bands: ['B4', 'B3', 'B2'],
  'bands': ['B8', 'B4', 'B3'],
}

rgb_vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

s2_ndwi_vis = {
  'min': -0.8,
  'max': 0.01,
  'palette': ['dfff7e','0000FF']
}

In [ ]:
# Select individual images to display on map
idx = 3
ndwi = ee.Image(ndwi_collection_list.get(idx)).clip(roi_s1)
print(date_list_ndwi[idx])


# Create map and add layers
M = geemap.Map()
M.centerObject(roi, 8)
M.add_basemap('HYBRID')
M.addLayer(ndwi,ndwi_vis,'Landsat NDWI')


idx1 = 44
s2_1 = ee.Image(s2_collection_list.get(idx1))
s2_ndwi1 = s2_1.normalizedDifference(['B3', 'B8']).rename('NDWI');

idx2 = idx1 + 1
s2_2 = ee.Image(s2_collection_list.get(idx2))
s2_ndwi2 = s2_2.normalizedDifference(['B3', 'B8']).rename('NDWI');

idx3 = idx1 + 2
s2_3 = ee.Image(s2_collection_list.get(idx3))
s2_ndwi3 = s2_3.normalizedDifference(['B3', 'B8']).rename('NDWI');

idx4 = idx1 + 3
s2_4 = ee.Image(s2_collection_list.get(idx4))
s2_ndwi4 = s2_4.normalizedDifference(['B3', 'B8']).rename('NDWI');


print(date_list_s2[idx1])
print(date_list_s2[idx2])
print(date_list_s2[idx3])
print(date_list_s2[idx4])


M.addLayer(s2_1,false_color_vis,'S2 False Color 1')
M.addLayer(s2_2,false_color_vis,'S2 False Color 2')
M.addLayer(s2_3,false_color_vis,'S2 False Color 3')
M.addLayer(s2_4,false_color_vis,'S2 False Color 4')

M.addLayer(s2_1,rgb_vis,'S2 RGB 1')
M.addLayer(s2_2,rgb_vis,'S2 RGB 2')
M.addLayer(s2_3,rgb_vis,'S2 RGB 3')
M.addLayer(s2_4,rgb_vis,'S2 RGB 4')

M.addLayer(s2_ndwi1,ndwi_vis,'S2 NDWI 1')
M.addLayer(s2_ndwi2,ndwi_vis,'S2 NDWI 2')
M.addLayer(s2_ndwi3,ndwi_vis,'S2 NDWI 3')
M.addLayer(s2_ndwi4,ndwi_vis,'S2 NDWI 4')


M  # show map

In [ ]:
basemaps = geemap.basemaps
for basemap in basemaps:
    print(basemap)
    
# M.add_basemap('HYBRID')    